# 🐘🎓 Spark & HDFS: Пошаговое руководство

Этот ноутбук создан, чтобы научить вас работать с **HDFS** и **Apache Spark** в интерактивном режиме.

## Чему мы научимся:
1.  Работать с HDFS через консольные команды прямо из Jupyter.
2.  Инициализировать SparkSession.
3.  Читать данные из HDFS (Parquet).
4.  Выполнять трансформации данных (Filter, GroupBy, Agg).
5.  Использовать SQL для анализа.
6.  Сохранять результаты обратно в HDFS.

---

## Шаг 1: Работа с HDFS (Консоль)

В Jupyter можно выполнять bash-команды, используя восклицательный знак `!`.
Давайте посмотрим, что у нас есть в файловой системе HDFS.

In [ ]:
# Проверим корневую директорию HDFS
!hdfs dfs -ls /

In [ ]:
# Проверим нашу папку пользователя, где лежат данные такси
!hdfs dfs -ls /user/myself/

---

## Шаг 2: Инициализация Spark

Чтобы работать с данными, нам нужно создать точку входа — `SparkSession`.
Мы указываем `master`, который соответствует нашему `spark-master` сервису в Docker.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, month, year, count, avg, desc

# Создаем сессию
spark = SparkSession.builder \
    .appName("SparkHDFSTutorial") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
spark

---

## Шаг 3: Чтение данных (Data Loading)

Spark умеет читать множество форматов: CSV, JSON, Parquet, Avro, ORC.
В нашем случае данные хранятся в эффективном формате **Parquet** (колоночное сжатие).

In [ ]:
# Читаем файл из HDFS
file_path = "hdfs://namenode:9000/user/myself/yellow_tripdata_2023-01.parquet"
df = spark.read.parquet(file_path)

# Выведем схему данных (типы колонок)
df.printSchema()

---

## Шаг 4: Основные операции с DataFrame

DataFrame API позволяет манипулировать данными функционально.

In [ ]:
# Выбор конкретных колонок (Select)
df.select("tpep_pickup_datetime", "passenger_count", "total_amount").show(5)

In [ ]:
# Фильтрация (Filter / Where)
# Найдем поездки, где стоимость больше $100
expensive_trips = df.filter(col("total_amount") > 100)

print(f"Количество дорогих поездок: {expensive_trips.count()}")
expensive_trips.select("tpep_pickup_datetime", "total_amount").show(5)

In [ ]:
# Группировка и Агрегация (GroupBy)
# Посчитаем средний чек для каждого VendorID
df.groupBy("VendorID") \
  .agg(
      count("*").alias("total_trips"),
      avg("total_amount").alias("avg_fare")
  ) \
  .show()

---

## Шаг 5: Использование SQL

Если вы знаете SQL, Spark позволяет использовать его напрямую. Для этого регистрируем таблицу.

In [ ]:
df.createOrReplaceTempView("taxi_trips")

In [ ]:
# Пример SQL-запроса: Топ 5 самых дорогих поездок
spark.sql("""
    SELECT 
        tpep_pickup_datetime, 
        passenger_count, 
        trip_distance, 
        total_amount
    FROM taxi_trips
    ORDER BY total_amount DESC
    LIMIT 5
""").show()

---

## Шаг 6: Запись результатов обратно в HDFS

Допустим, мы сделали агрегацию и хотим сохранить результат.

In [ ]:
# Подготовим маленький DataFrame с результатами
summary_df = df.groupBy("payment_type").count()

# Сохраним его в CSV в HDFS
output_path = "hdfs://namenode:9000/user/myself/payment_summary"

# mode("overwrite") перезапишет папку, если она существует
summary_df.write.mode("overwrite").csv(output_path)

print("Данные сохранены!")

In [ ]:
# Проверим через HDFS CLI, что файлы создались
!hdfs dfs -ls /user/myself/payment_summary

---

## 🎉 Вывод

Вы прошли полный цикл работы Data Engineer'а:
1.  Проверили данные в хранилище (HDFS).
2.  Загрузили их в движок обработки (Spark).
3.  Проанализировали (SQL/DataFrame).
4.  Сохранили результат (ETL).

Теперь вы можете загружать свои файлы и экспериментировать!

In [ ]:
spark.stop()